<h1><center>Practical: Tomography for binary pictures</center></h1>
<h3><center>Dominic Therattil (S4228952) & Sebastian Preststulen (S4270754)</center></h3>


<h2>Assignment 1 - Computation of the horizontal and vertical projections (30)</h2>

In [ ]:
import cv2

imname = 'rectangle'
impath = 'Practical_Tomography/' + imname + '.png'
A = cv2.imread(impath)

The 2D array A now contains the pixel values of the picture *rectangle.png*. Let n be the number of rows of A. The number of columns of A is n as well (check this).

- (15pt) Compute the column sums of A, and store the result in a vector *colsum*, which will have n elements. Compute the row sums of A, and store the result in a vector *rowsum*, which also will have n elements.

In [ ]:
white = [255, 255, 255]
rowsum = [0] * len(A)
colsum = [0] * len(A)

for i in range (len(A)):
    for j in range (len(A)):
        #computing column sums
        if ((A[j][i] == white).all()):
            colsum[i] += 1
        #computing row sums
        if ((A[i][j] == white).all()):
            rowsum[i] += 1

- (10pt) Save the column and row sums in a file <em>rectangle_HVproj.mat</em> via the commands:

In [ ]:
import scipy.io

matlab_result = {"colsum": colsum, "rowsum": rowsum}

projfile = imname + '_HVproj' + '.mat'
scipy.io.savemat(projfile, matlab_result)

The *_HVproj* label indicates that you only computed horizontal and vertical projections.

- (5pt) Use your program to save the column and row sums of the picture *oval.png* to a file, and do thesame for *rock.png*. Save your program in a file with name *projections_HV.m*. If you use the live script, you may directly place your code inside the block below:

In [ ]:
import cv2, scipy.io

imname = 'rock'
impath = 'Practical_Tomography/' + imname + '.png'

A = cv2.imread(impath)

white = [255, 255, 255]
rowsum = [0] * len(A)
colsum = [0] * len(A)

for i in range (len(A)):
    for j in range (len(A)):
        #computing column sums
        if ((A[j][i] == white).all()):
            colsum[i] += 1
        #computing row sums
        if ((A[i][j] == white).all()):
            rowsum[i] += 1

matlab_result = {"colsum": colsum, "rowsum": rowsum}

projfile = imname + '_HVproj' + '.mat'
scipy.io.savemat(projfile, matlab_result)

In [ ]:
import cv2, scipy.io

imname = 'oval'
impath = 'Practical_Tomography/' + imname + '.png'

A = cv2.imread(impath)

white = [255, 255, 255]
rowsum = [0] * len(A)
colsum = [0] * len(A)

for i in range (len(A)):
    for j in range (len(A)):
        #computing column sums
        if ((A[j][i] == white).all()):
            colsum[i] += 1
        #computing row sums
        if ((A[i][j] == white).all()):
            rowsum[i] += 1

matlab_result = {"colsum": colsum, "rowsum": rowsum}

projfile = imname + '_HVproj' + '.mat'
scipy.io.savemat(projfile, matlab_result)

<h2>Assignment 2 - Reconstruction from horizontal and vertical projections (70)</h2>

Now you will implement the Kacmarz reconstruction algorithm for binary images in Python. Call the file with your implementation *tomography_HV.m*.
Start by loading the projection data of the picture <em>rectangle.png</em> via the commands:


In [ ]:
import scipy.io

imname = 'rectangle'
projfile = imname + '_HVproj' + '.mat'
matlab_result = scipy.io.loadmat(projfile)

The variables *colsum* and *rowsum* now contain the column and row sums of the picture *rectangle.png*.

- (30pt) Implement Algorithm 2.2 from the syllabus in Matlab. We use the matrix representation of a picture. To test your program it can be useful to take a small n × n-matrix A as input. You can generate such a matrix with random zeroes and ones via the rand function, e.g.: 

In [ ]:
import numpy as np

n = 4
random_matrix_array = np.random.uniform(low=0, high=0.5, size=(n,n))

- We chose to operate using normal arrays and convert our final computed array into a numpy array when we need to process it into an image

For the initialization you can use an n × n matrix f(0) with zeroes. Initially set MAX_ITER to 1. When your program is complete you can use a value for MAX_ITER in the range 100-200.

In [ ]:
import scipy.io, cv2, numpy as np

imname = 'rectangle'
projfile = imname + '_HVproj' + '.mat'
matlab_result = scipy.io.loadmat(projfile)

colsum = matlab_result['colsum'][0]
rowsum = matlab_result['rowsum'][0]
n = len(colsum)
MAX_ITER = 50
eps = 0.00000005

#initialising empty matrixes
f0 = np.zeros((n,n))
fn = np.zeros((n,n))
finit = np.zeros((n,n))

# kaczmarz method for reconstruction
for k in range(MAX_ITER):
    #setting f0
    f0 = finit.copy()
    
    #looping through columns
    Bc = np.zeros(n)
    for c in range(n):
        Bc[c] = (sum(f0[:,c]) - colsum[c])/n
        fn[:,c] = f0[:,c] - Bc[c]
    #correcting for pos and neg values
    fn[fn<0]=0            
    fn[fn>1]=1  

    #setting f0
    f0 = fn
    #looping through rows
    Br = np.zeros(n)
    for r in range(n):
        Br[r] =  (sum(f0[r,:]) - rowsum[r])/n
        fn[r,:] = f0[r,:] - Br[r]
    #correcting for pos and neg values
    fn[fn<0]=0            
    fn[fn>1]=1   

    #comparing our reconstructed matrix with accuracy threshold   
    relDif = abs(np.matrix(fn).sum() - np.matrix(finit).sum() ) / (n*n)
    if (relDif < eps):
        break

    #setting finit
    finit = fn.copy()

#converting 1 values to white rgb value
np.matrix(fn).round(0,fn)
fn[fn==1]=255

Tips:

- Algorithm 2.2 contains a for-loop over the n column sums and a for-loop over the n row sums. First implement one of these for-loops.
- The essential step concerns lines 8-9 and 14-15, where the update of the current approximation of the solution takes place.
- The essential step concerns lines 8-9 and 14-15, where the update of the current approximation of the solution takes place. <u>Avoid extra loops as much as possible</u>. Use the vectorisations of Numpy: you can compute all row or column sums in one go by the routine sum. A complete column or row i of a matrix f you obtain via *f(:, i)* or *f(i, :)*, respectively. Subtracting the same number β from a complete row or column can be easily done via the function ones(). Use Google.
- The constraints in line 10 and 16 can be enforced by applying the Python functions min and max to a complete row or column.
- The double summation in line 18 can be computed easily (that is, without loops) by using the Matlab sum and abs functions.
- To follow the progression during the iteration process it is convenient to add a print statement after line 20, for example:

In [ ]:
#Open file at the beginning of script
filename = open("filename", 'w')

print('k =', k, 'delta =', delta, file=filename)

#Close file when no longer needed
filename.close()

- where delta is the value of the double sum in line 18.

- (5pt) Apply your program to the picture rectangle.png, and use the *imwrite* function to save the result as a picture in PNG format. You might also add the final value of k and the used value of epsilon to the file name, e.g.,
  
- This is convenient when you carry out several experiments with different values of these parameters.

In [ ]:
#saving recreated image
recfile = imname + '_HVrec_' + '_k=' + str(k) + '_eps=' + str(eps) + '.png'
cv2.imwrite(recfile, fn)

- (5pt) Add a line to your code in which the relative error *E* between the input array A and the reconstruction *f* is computed, that is, the quantity

$E = ||f^{(k^{*})} - A|| := \frac{1}{n^{2}}\sum_{r = 1}^{n}\sum_{c = 1}^{n} |f_{r,c}^{k^{*}} - A_{r,c}|$

where $k^{*}$ is the value of the loop variable k at which the algorithm terminated, and $f^{k^{*}}$ is the corresponding reconstruction. 

In [ ]:
impath = 'Practical_Tomography/' + imname + '.png'
A = cv2.imread(impath)
A = A[:,:,0]
E = abs(np.matrix(fn).sum() - np.matrix(A).sum() ) / (n*n)
print(E)

- (5pt) To see at which pixel locations the differences between input and reconstruction occur it is convenient to compute a difference picture $dif = f^{k^{*}} - A$. Add a line to your code to do this and save the result again as a picture in PNG format, for example as follows:

In [ ]:
imname = "rectangle"
impath = 'Practical_Tomography/' + imname + '.png'
A = cv2.imread(impath)
A = A[:,:,0]
dif = fn - A

diffile = imname + '_HVdif_' + '_k=' + str(k) + '_eps=' + str(eps) + '.png'
cv2.imwrite(diffile, dif)

- (5pt) Repeat the steps above for the pictures *oval.png* and *rock.png*.

In [ ]:
# difference image for oval
imname = "oval"
impath = 'Practical_Tomography/' + imname + '.png'
A = cv2.imread(impath)
A = A[:,:,0]
dif = fn - A

diffile = imname + '_HVdif_' + '_k=' + str(k) + '_eps=' + str(eps) + '.png'
cv2.imwrite(diffile, dif)

#difference image for rock
imname = "rock"
impath = 'Practical_Tomography/' + imname + '.png'
A = cv2.imread(impath)
A = A[:,:,0]
dif = fn - A

diffile = imname + '_HVdif_' + '_k=' + str(k) + '_eps=' + str(eps) + '.png'
cv2.imwrite(diffile, dif)

- (20pt) Describe your experiments, the values of the parameters that were used, and the relative error values obtained. For each of the input pictures, include three pictures in a row in your report: the input picture, the reconstruction picture, and the difference picture. Discuss the results. Explain the differences between the reconstruction quality of the various input pictures.

Original                                |Reconstruction             |                Difference       
:-------------------------:             |:-------------------------:|                :-------------------------:
![](Practical_Tomography/rectangle.png) |  ![](rectangle_HVrec__k=60_eps=5e-08.png) |![](rectangle_HVdif__k=60_eps=5e-08.png)

By simply viewing the images we can see there is not much difference between the original image and the reconstructed image, this is reaffirmed by looking at the difference image where we see some scattered white dots representing the difference in pixels between the original and reconstruction. Furthermore, altering the MAX_ITER from 1, 10,50,100 at an accuracy threshold of 0.00000005 after 60 iterations we found that we reached our accuracy threshold and any furthere iteration would not provide signifcant change to our reconstruction.

Original                                |Reconstruction             |                Difference       
:-------------------------:             |:-------------------------:|                :-------------------------:
![](Practical_Tomography/rock.png) |  ![](rock_HVrec__k=499_eps=5e-08.png) |![](rock_HVdif__k=499_eps=5e-08.png)

Above see our 2 representations of the rock and the difference. First, we have the original picture, the reconstructed picture and finally the result from finding the difference between the original and reconstructed image. At first glance, and just by looking at the 2 pictures, we can see that there is a very minimal difference between the two. The only noticeable difference is at the bottom and the top of the two photos, where we see more black pixels in the original than in the reconstructed image. This is shown in the difference result, as we can see the white lining at the bottom and the top of the result. We also see white lines on the bottom right and top left running along the edge of the rock, which indicates to us that the reconstructed image did not succeed in including these.

Furthermore, upon testing different values for the the MAX_ITER at an accuracy threshold of 0.00000005 we found that when MAX_ITER = 500 and the reconstruction image was the best, it had minimal whitespace and had the smallest difference E value between the original image.

Original                                |Reconstruction             |                Difference       
:-------------------------:             |:-------------------------:|                :-------------------------:
![](Practical_Tomography/oval.png) |  ![](oval_HVrec__k=112_eps=5e-08.png) |![](oval_HVdif__k=112_eps=5e-08.png)

Examining the orginal image and reconstruction image we see there is quite a drastic difference, this is further reaffirmed by white spaces in the difference image. Inaddition, When we examing the reconstruction with different MAX_ITER at an accuracy threshold of 0.00000005 we see reach our accuracy threshold after 112 iterations and any further iterations would not provide any significant change in our reconstruction
